# NLP - lab2

### Mateusz Praski

---


In [1]:
import requests
import json
import sys

import numpy as np
import pandas as pd

from scipy.sparse import csr_matrix
from glob import glob
from tqdm import trange, tqdm
from sklearn.metrics import ndcg_score

# Task 3 & 4
Define an ES analyzer for Polish texts containing:
- standard tokenizer
- synonym filter with alternative forms for months, e.g. wrzesień, wrz, IX.
- lowercase filter
- Morfologik-based lemmatizer
- lowercase filter (looks strange, but Morfologi produces capitalized base forms for proper names, so we have to lowercase them once more).

Define another analyzer for Polish, without the synonym filter.

In [2]:
analyzer_with_synonyms = {
    'type': 'custom',
    'tokenizer': 'standard',
    'filter': [
        'months-synonyms',
        'lowercase',
        'morfologik_stem',
        'lowercase'
    ]
}

analyzer_without_synonyms = {
    'type': 'custom',
    'tokenizer': 'standard',
    'filter': [
        'lowercase',
        'morfologik_stem',
        'lowercase'
    ]
}

no_lemma_synonyms = {
    'type': 'custom',
    'tokenizer': 'standard',
    'filter': [
        'months-synonyms',
        'lowercase',
    ]
}

no_lemma_no_synonyms = {
    'type': 'custom',
    'tokenizer': 'standard',
    'filter': [
        'lowercase',
    ]
}

filters = {
    'months-synonyms': {
        'type': 'synonym',
        'synonyms': [
            'sty, I => styczeń',
            'lut, II => luty',
            'mar, III => marzec',
            'kwi, IV => kwiecień',
            'V => maj',
            'cze, VI => czerwiec',
            'lip, VII => lipca',
            'sie, VIII => sierpnia',
            'wrz, IX => wrzesień',
            'paz, X => pażdziernik',
            'lis, XI => listopad',
            'gru, XII => grudzień'
        ]
    }
}

# Task 5

Define an ES index for storing the contents of the corpus from lab 1 using both analyzers. Use different names for the fields analyzed with a different pipeline.

In [3]:
index_definition = {
    'mappings': {
        'properties': {
            'answer': {
                'type': 'text',
                'fields': {
                    'with_synonyms': {
                        'type': 'text',
                        'analyzer': 'analyze_with_synonyms'
                    },
                    'without_synonyms': {
                        'type': 'text',
                        'analyzer': 'analyze_without_synonyms'
                    }
                }
            },
        }
    },
    'settings': {
        'analysis': {
            'analyzer': {
                'analyze_with_synonyms': analyzer_with_synonyms,
                'analyze_without_synonyms': analyzer_without_synonyms,
                'analyze_no_lemma_synonyms': no_lemma_synonyms,
                'analyze_no_lemma_no_synonyms': no_lemma_no_synonyms
            },
            'filter': filters
        }
    }
}

In [4]:
body = json.dumps(index_definition, indent=4)
print(body)

{
    "mappings": {
        "properties": {
            "answer": {
                "type": "text",
                "fields": {
                    "with_synonyms": {
                        "type": "text",
                        "analyzer": "analyze_with_synonyms"
                    },
                    "without_synonyms": {
                        "type": "text",
                        "analyzer": "analyze_without_synonyms"
                    }
                }
            }
        }
    },
    "settings": {
        "analysis": {
            "analyzer": {
                "analyze_with_synonyms": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "months-synonyms",
                        "lowercase",
                        "morfologik_stem",
                        "lowercase"
                    ]
                },
                "analyze_without_synonyms": {
                    "typ

In [ ]:
!docker cp elastic_container:/usr/share/elasticsearch/config/certs/http_ca.crt .

In [5]:
ELASTIC_IP = 'https://localhost:9200'
INDEX = 'nlp-index'
auth = ('elastic', '"qwerty"')
cert = 'http_ca.crt'

In [7]:
res = requests.delete(f'{ELASTIC_IP}/{INDEX}', auth=auth, verify=cert)
res.status_code, print(res.content.decode())

{"acknowledged":true}


(200, None)

In [8]:
res = requests.put(f'{ELASTIC_IP}/{INDEX}', json=index_definition, auth=auth, verify=cert)
res.status_code, print(res.content.decode())

{"acknowledged":true,"shards_acknowledged":true,"index":"nlp-index"}


(200, None)

In [6]:
df = pd.read_json("../../data/corpus.jsonl", lines=True)
df = df.set_index('_id').sort_index()
df.head()

,title,text,metadata
_id,,,
3,,"Nie mówię, że nie podoba mi się też pomysł szk...",{}
31,,Tak więc nic nie zapobiega fałszywym ocenom po...,{}
56,,Nigdy nie możesz korzystać z FSA dla indywidua...,{}
59,,Samsung stworzył LCD i inne technologie płaski...,{}
63,,Oto wymagania SEC: Federalne przepisy dotycząc...,{}


# Task 6

Load the data to the ES index.

In [7]:
def bulk_insert(rows, col):
    n = len(rows)
    request_command = [f'{{"index": {{ "_index": "{INDEX}" }} }}'] * n
    request_data = [f'{{"with_synonyms": {text}, "without_synonyms": {text}}}' for text in rows]

    payload = [None] * (n * 2)
    payload[::2] = request_command
    payload[1::2] = request_data
    body = "\n".join(payload) + '\n'
    print(body)
    # Didn't work  in the end :(
    res = requests.post(f'{ELASTIC_IP}/_bulk?pretty', data=body, headers={'Content-type': 'application/x-ndjson'}, auth=auth, verify=cert)
    return res

In [8]:
def standard_insert(df, disable_tqdm=False):
    for index, row in tqdm(df.iterrows(), disable=disable_tqdm, total=len(df.index)):
        rs = requests.put(f"{ELASTIC_IP}/{INDEX}/_doc/{index}", json={"answer": row['text']}, auth=auth, verify=cert)
        if rs.status_code != 201 and rs.status_code != 200:
            raise RuntimeError(f"{rs.status_code} - {rs.text}")

In [25]:
# res = requests.post(f"{ELASTIC_IP}/{INDEX}/_doc", json={"text": text}, auth=auth, verify=cert)

In [26]:
# res.status_code

In [27]:
standard_insert(df)

100%|██████████| 57638/57638 [12:39<00:00, 75.94it/s]


In [8]:
requests.get(f"{ELASTIC_IP}/{INDEX}/_count", auth=auth, verify=cert).text

'{"count":57638,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0}}'

In [32]:
res = requests.get(f"{ELASTIC_IP}/{INDEX}/_search?pretty&size=5", auth=auth, verify=cert, json={
    "query": {
        "multi_match": {
            "query": df.iloc[0]['text'],
            "fields": [
                "answer.with_synonyms"
            ]
        }
    }
})

In [9]:
def query_text(text, analyzer=None, limit=None, use_synonyms=True):
    url = f"{ELASTIC_IP}/{INDEX}/_search"
    if limit is not None:
        url += f"?size={limit}"

    index = "answer.with_synonyms" if use_synonyms else "answer.without_synonyms"

    body  = {
        "query": {
            "match": {
                index: {
                    "query": text,
                }
            }
        }
    }

    if analyzer is not None:
        body["query"]["match"][index]["analyzer"]  = analyzer

    rs = requests.get(url, json=body, auth=auth, verify=cert)
    if rs.status_code != 200:
        raise RuntimeError(f"{rs.status_code} - {rs.text}")
    return json.loads(rs.text)

# Task 7

Determine the number of documents containing the word `styczeń` (in any form) including and excluding the synonyms.

## With synonyms

In [10]:
query_text("styczeń")['hits']['total']['value']

3101

## Without synonyms

In [11]:
query_text("styczeń", use_synonyms=False)['hits']['total']['value']

329

It's worth mentioning that synonym for `styczeń` is  `I`, which may occur in some other texts

In [12]:
query_text("styczeń")['hits']['hits'][0]

{'_index': 'nlp-index',
 '_id': '71552',
 '_score': 5.8238015,
 '_source': {'answer': 'Niech P oznacza kwotę inwestycji, R stopę zwrotu, a I stopę inflacji. Dla uproszczenia załóżmy, że płatność p jest dokonywana corocznie zaraz po uzyskaniu zwrotu. Tak więc, na koniec roku, inwestycja P wzrosła do P*(1+R), a p jest zwracane jako wypłata renty. Jeżeli I = 0, cały zwrot może zostać wypłacony jako zapłata, a więc p = P*R. Oznacza to, że pod koniec roku, gdy kurz opadnie po odebraniu zwrotu P*R i wypłaceniu go jako renty dożywotniej, P jest ponownie dostępne na początku następnego roku, aby zarobić zwrot według stawki R. My mieć P*(1+R) - p = P Jeżeli I > 0, to na koniec roku, po opadnięciu kurzu, nie możemy sobie pozwolić na posiadanie tylko P jako inwestycji na przyszły rok. Przyszłoroczna opłata musi wynosić p*(1+I), więc potrzebujemy większej inwestycji, ponieważ stopa zwrotu jest stała. O ile większy? Cóż, jeśli inwestycja na początku przyszłego roku wyniesie P*(1+I), zarobi dokładni

# Task 9
Compute NDCG@5 for the QA dataset (the test subset) for the following setups
- synonyms enabled and disabled,
- lemmatization in the query enabled and disabled.

In [13]:
questions = pd.read_json('../../data/queries.jsonl', lines=True)
questions.head()

,_id,text,metadata
0,0,Co jest uważane za wydatek służbowy w podróży ...,{}
1,4,Wydatki służbowe - ubezpieczenie samochodu pod...,{}
2,5,Rozpoczęcie nowego biznesu online,{}
3,6,„Dzień roboczy” i „termin płatności” rachunków,{}
4,7,Nowy właściciel firmy – Jak działają podatki d...,{}


In [14]:
# qa = pd.concat([
#     pd.read_csv(path, sep='\t')
#     for path in glob('../../data/*.tsv')
# ])
qa = pd.read_csv('../../data/test.tsv', sep='\t')

qa = qa.sort_values(by='query-id')
qa.head()

,query-id,corpus-id,score
0,8,566392,1
1,8,65404,1
2,15,325273,1
3,18,88124,1
4,26,285255,1


In [15]:
questions = questions[questions['_id'].isin(qa['query-id'])].reset_index()
questions.head()

,index,_id,text,metadata
0,6000,4641,Gdzie powinienem zaparkować mój fundusz na des...,{}
1,6001,5503,Względy podatkowe związane ze sprzedażą rodzin...,{}
2,6002,7803,Czy Delta może być wykorzystana do obliczenia ...,{}
3,6003,7017,Podstawowa strategia handlu algorytmicznego,{}
4,6004,10152,"Co oznacza dla firmy wysoka marża operacyjna, ...",{}


In [16]:
qa['score'].nunique()

1

In [17]:
qa_mapping = csr_matrix(
    (qa['score'], (qa['query-id'], qa['corpus-id'])),
    shape=(qa['query-id'].max() + 1, df.index.max() + 1),
    dtype=int
)

In [18]:
qa_mapping[1, 2]

0

In [19]:
max_matches = qa.groupby('query-id')['corpus-id'].count().rename('count')
max_matches

query-id
8        2
15       1
18       1
26       2
34       1
        ..
10979    4
10994    2
11039    8
11054    2
11088    1
Name: count, Length: 648, dtype: int64

In [20]:
def eval_answers(questions, analyzer, use_syonynms):
    no_questions = len(questions.index)
    rec = np.empty((no_questions, 5), dtype=int)

    for index, row in tqdm(questions.iterrows(), total=no_questions):
        rs = query_text(row['text'], analyzer=analyzer, limit=5, use_synonyms=use_syonynms)

        recs = [qa_mapping[int(row['_id']), int(rs['_id'])] for rs in rs['hits']['hits'][:5]]
        if len(recs) < 5:
            recs += [0] * (5 - len(recs))

        rec[index] = recs
    return rec

In [21]:
rec_lemma_synonyms = eval_answers(questions, 'analyze_with_synonyms', use_syonynms=True)

100%|██████████| 648/648 [00:09<00:00, 65.60it/s]


In [22]:
rec_lemma_no_synonyms = eval_answers(questions, 'analyze_without_synonyms', use_syonynms=False)

100%|██████████| 648/648 [00:09<00:00, 67.12it/s]


In [23]:
rec_no_lemma_synonyms = eval_answers(questions, 'analyze_no_lemma_synonyms', use_syonynms=True)

100%|██████████| 648/648 [00:08<00:00, 79.26it/s]


In [24]:
rec_no_lemma_no_synonyms = eval_answers(questions, 'analyze_no_lemma_no_synonyms', use_syonynms=False)

100%|██████████| 648/648 [00:08<00:00, 75.80it/s]


In [25]:
perfect_answers = np.zeros((len(questions.index), 5), dtype=int)

for index, row in tqdm(questions.iterrows()):
    matches = min(max_matches.loc[row['_id']], 5)
    vector = ([1] * matches) + ([0] * (5 - matches))
    perfect_answers[index, :] = vector

648it [00:00, 29178.39it/s]


In [28]:
def eval_ndcg(y_true, y_pred, return_mean=True):
    n = y_true.shape[1]
    no_questions = y_true.shape[0]

    dcg_weights = np.log2(np.arange(2, n+2))
    dcg_weights = np.resize(dcg_weights, (no_questions, n))
    dcg = np.sum(y_pred / dcg_weights, axis=1)
    idcg = np.sum(y_true / dcg_weights, axis=1)
    ndcg = dcg / idcg

    return ndcg.mean() if return_mean else ndcg

In [29]:
pd.DataFrame(data={
    "lemmatization": [
        eval_ndcg(perfect_answers, rec_lemma_synonyms),
        eval_ndcg(perfect_answers, rec_lemma_no_synonyms)
    ],
    "no_lemmatization": [
        eval_ndcg(perfect_answers, rec_no_lemma_synonyms),
        eval_ndcg(perfect_answers, rec_no_lemma_no_synonyms)
    ]},
    index=["synonyms", "no_synonyms"]
).applymap(lambda x: f"{x:.2%}")

,lemmatization,no_lemmatization
synonyms,18.51%,7.96%
no_synonyms,18.51%,7.96%


## 11. Question where the relevant document is returned by ES at position 1,

In [59]:
q_id = np.where(rec_lemma_no_synonyms[:, 0] == 1)
q = questions.iloc[q_id]['text'].iloc[0]
q

'Jaka jest różnica między „handlowcem” a „maklerem giełdowym”?'

In [61]:
[x['_source']['answer'] for x in query_text(q, limit=5)['hits']['hits']]

['Handlowcy zarabiają na życie, maklerzy giełdowi mówią ludziom, aby angażowali się w transakcje na życie. Aby zostać zatrudnionym jako trader, potrzebujesz udokumentowanego doświadczenia w zakresie konsekwentnego zarabiania pieniędzy. Aby być zatrudnionym jako makler giełdowy, musisz uzyskać licencję, ale nie musisz udowadniać, że możesz konsekwentnie zarabiać pieniądze.',
 "„Poniżej jest tylko trochę informacji na temat krótkiej sprzedaży z mojej małej, unikalnej książki „„Mały trader giełdowy””: Krótka sprzedaż to zaawansowane narzędzie do handlu akcjami z unikalnym ryzykiem i korzyściami. Jest to przede wszystkim krótkoterminowa strategia handlowa o charakterze technicznym natury, głównie dokonywanej przez małych maklerów giełdowych, animatorów rynku i fundusze hedgingowe. Większość małych maklerów giełdowych używa głównie krótkiej sprzedaży jako narzędzia do krótkoterminowych spekulacji, gdy czują, że cena akcji jest nieco zawyżona. Większość długoterminowych pozycji krótkich jest

This question contains very specific words ('handlowiec', 'markler giełdowy'), which may be very rare in dataset, making it much easier to find it

# Question where the relevant document is returned by ES at position 4 or 5.

In [62]:
q_id = np.where(rec_lemma_no_synonyms[:, 4] == 1)
q = questions.iloc[q_id]['text'].iloc[0]
q

'Dlaczego stacje benzynowe pobierają różne kwoty w tej samej okolicy?'

In [69]:
[x['_source']['answer'] for x in query_text(q, limit=5)['hits']['hits']]

['W mojej okolicy jest wiele stacji benzynowych, które mają już różne ceny, jeśli płacisz gotówką lub kredytem. Ponadto robią to również niektóre małe firmy. Moja żona kupiła w piekarni tort urodzinowy. Jeśli zapłaciłeś gotówką, zaoszczędziłeś 5%.',
 'Jest wiele czynników. Większość stacji benzynowych ustala ceny gazu na podstawie tego, ile będzie kosztować jego wymiana. Kiedy więc ich dostawca podnosi cenę, którą pobiera od stacji, stacja zazwyczaj podnosi proporcjonalnie swoje ceny. Dostawcy mają zwykle własne stawki. Firma musi osiągać zyski, więc ustala cenę tam, gdzie czuje, że zarobi najwięcej pieniędzy. Niektóre stacje kupują po okazyjnej cenie gaz. Wiele osób twierdzi, że ten gaz jest w porządku. Osobiście wydaje się, że na niektórych stacjach benzynowych moje samochody jeżdżą znacznie gorzej. Mogę powiedzieć, że mój przebieg może się różnić nawet o 4 mile do galonu w zależności od tego, skąd biorę benzynę. Więc płacę więcej, aby jechać do tych stacji, które konsekwentnie dosta

This question contains generic words, which may result finding other texts related to gas stations, but not specifically to this question.

# Question where the relevant document is returned by ES not found.

In [71]:
q_id = np.where(rec_lemma_no_synonyms.sum(axis=1) == 0)
q = questions.iloc[q_id]['text'].iloc[1]
q

'Względy podatkowe związane ze sprzedażą rodzinie nieruchomości poniżej szacunkowej wartości?'

In [72]:
[x['_source']['answer'] for x in query_text(q, limit=5)['hits']['hits']]

['Musisz określić jurysdykcję podatkową, kiedy zostanie wykonana następna wycena podatkowa. W niektórych przypadkach wycena jest przeprowadzana co rok, dwa lub trzy lata. W innych przypadkach odbywa się to również przy sprzedaży nieruchomości. Informacje te powinny znaleźć się na stronie powiatowego urzędu skarbowego. Będą również mieli informacje o tym, jak się odwołać. Większość jurysdykcji ma sposób sprawdzania nie tylko stawek, ale także wartości danej nieruchomości. Będziesz także zainteresowany ustaleniem, czy wartość podatkowa nieruchomości jest niższa ze względu na przepisy lokalne/stanowe, które ograniczają wzrost wartości z jednej oceny do drugiej. Sprzedaż nieruchomości może wywołać nadrabianie zaległości. Możliwe, że obniżona wartość domu została już uwzględniona w ocenie. Możliwe też, że nie. Należy pamiętać, że podatki w niektórych jurysdykcjach mogą być bardziej skomplikowane, ponieważ istnieje również składnik miasto/miasto/hrabstwo, a w niektórych miejscach również inn

In [74]:
q_id = np.where(rec_lemma_no_synonyms.sum(axis=1) == 0)
questions.iloc[q_id].iloc[1]

level_0                                                     1
index                                                    6001
_id                                                      5503
text        Względy podatkowe związane ze sprzedażą rodzin...
metadata                                                   {}
Name: 1, dtype: object

In [75]:
qa[qa['query-id'] == 5503]

,query-id,corpus-id,score
978,5503,146277,1
979,5503,64279,1


In [73]:
[x['_id'] for x in query_text(q, limit=5)['hits']['hits']]

['61623', '85229', '497530', '407433', '218622']

This question also contains common words ('podatki', 'sprzedaż', 'rodzinie'), but it's very specific (question about taxation in some case)

In [44]:
np.savez(
    'rs/scores2.npz',
    no_lemma_no_synonyms=rec_no_lemma_no_synonyms,
    no_lemma_synonyms=rec_no_lemma_synonyms,
    lemma_no_synonyms=rec_lemma_no_synonyms,
    lemma_synonyms=rec_lemma_synonyms
)

# Questions and answers

### What are the strengths and weaknesses of regular expressions versus full text search regarding processing of text?

FTS allows for text query using natural language, in comparison to regular expression, which takes time to write and debug, even for simple searches (e.g. `january` in polish). They're also very customizable, and easy to deliver to end user.

On the other hand, they do not allow for the same granularity of the query as regular expressions, and require bigger setup (Elastic container + API + security setup vs any programming language in the world)

---

### Is full text search applicable to the question answering problem? Show at least 3 examples from the corpus to support your claim.

They might be useful, as some kind of initial solution, but in case of questions and answers they might require usage knowledge from the end user (there's a reason why some people claim googling to be a skill). 

According to NDCG@5 scores, FTS managed to get around 70% of the answers right. But those solutions might fail in some cases, as we have seen in earlier questions and answers